In [ ]:
!pip install scipy dask matplotlib

In [ ]:
from pathlib import Path
from zipfile import ZipFile

import httpx
import rioxarray
import scipy.ndimage
from rich.progress import track

from jupytergis.tiler import GISDocument

We download the [HydroSHEDS](https://hydrosheds.org) dataset, and in particular the digital elevation model and the flow accumulation for South America. You can think of flow accumulation as a potential river flow, so we will try to visualize rivers on top of the terrain.

In [ ]:
url = "https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/hydrosheds/sa_30s_zip_grid/sa_acc_30s_grid.zip"
filename = Path(url).name
name = filename[: filename.find("_grid")]
adffile_acc = Path(name) / name / "w001001.adf"

if not adffile_acc.exists():
    with httpx.stream("GET", url) as r, open(filename, "wb") as f:
        total = int(r.headers["Content-Length"]) / 1024
        for data in track(r.iter_bytes(chunk_size=1024), total=total, description="Downloading"):
            f.write(data)
        f.flush()
    zip = ZipFile(filename)
    zip.extractall(".")

In [ ]:
url = "https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/hydrosheds/sa_30s_zip_grid/sa_dem_30s_grid.zip"
filename = Path(url).name
name = filename[: filename.find("_grid")]
adffile_dem = Path(name) / name / "w001001.adf"

if not adffile_dem.exists():
    with httpx.stream("GET", url) as r, open(filename, "wb") as f:
        total = int(r.headers["Content-Length"]) / 1024
        for data in track(r.iter_bytes(chunk_size=1024), total=total, description="Downloading"):
            f.write(data)
        f.flush()
    zip = ZipFile(filename)
    zip.extractall(".")

Let's load the data into data arrays.

In [ ]:
da_acc = rioxarray.open_rasterio(adffile_acc, masked=True)
da_acc

In [ ]:
da_dem = rioxarray.open_rasterio(adffile_dem, masked=True)
da_dem

We just need to select the band. We will also chunk it in order to improve performances.

In [ ]:
da_acc = da_acc.sel(band=1)
da_acc = da_acc.chunk(dict(x=1000, y=1000))

In [ ]:
da_dem = da_dem.sel(band=1)
da_dem = da_dem.chunk(dict(x=1000, y=1000))

We will need the min/max values to rescale.

In [ ]:
vmin_acc, vmax_acc = int(da_acc.min().compute()), int(da_acc.max().compute())
vmin_acc, vmax_acc

In [ ]:
vmin_dem, vmax_dem = int(da_dem.min().compute()), int(da_dem.max().compute())
vmin_dem, vmax_dem

In [ ]:
from pathlib import Path

import numpy as np
from rio_tiler.models import ImageData
from titiler.core.algorithm import BaseAlgorithm

radius = 1
circle = np.zeros((2 * radius + 1, 2 * radius + 1)).astype("uint8")
y, x = np.ogrid[-radius : radius + 1, -radius : radius + 1]
index = x**2 + y**2 <= radius**2
circle[index] = 1


class AlgoAcc(BaseAlgorithm):
    def __call__(self, img: ImageData) -> ImageData:
        data = np.log(img.data[0])
        data = scipy.ndimage.maximum_filter(data, footprint=circle)
        mask = np.where((np.isnan(data)) | (data < np.log(1000)), 255, 0)
        data = np.ma.MaskedArray(data, mask=mask)
        return ImageData(
            data,
            assets=img.assets,
            crs=img.crs,
            bounds=img.bounds,
        )

In [ ]:
Path("south_america.jGIS").unlink(missing_ok=True)
doc = GISDocument("south_america.jGIS")
doc

In [ ]:
await doc.add_tiler_layer(
    name="Digital elevation model layer",
    data_array=da_dem,
    colormap_name="terrain",
    rescale=(vmin_dem, vmax_dem),
)

In [ ]:
await doc.add_tiler_layer(
    name="Flow accumulation layer",
    data_array=da_acc,
    colormap_name="plasma",
    rescale=np.log([vmin_acc, vmax_acc]),
    algorithm=AlgoAcc,
    reproject="max",
)

In [ ]:
# not mandatory, but helps shutting down the kernel gracefully:
# await doc.stop_tile_server()